In [1]:
import yfinance as yf
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
start = dt.datetime(2011, 1, 1)
starting_date = dt.datetime(2012, 1, 1)
end = dt.datetime.now()
interval = '1mo'

df = yf.download('NVDA', start=start, end=end, interval=interval)

[*********************100%%**********************]  1 of 1 completed


In [3]:
data = df[['Adj Close']].copy()
data['returns'] = data['Adj Close'].pct_change()

In [4]:
def lagit(df, lags):
    names = []
    for i in range(3, lags+1, 3):
        df['Lag_'+str(i)] = df['returns'].shift(i)
        names.append('Lag_'+str(i))
    return names

In [5]:
lagnames = lagit(data, 9)
data

,Adj Close,returns,Lag_3,Lag_6,Lag_9
Date,,,,,
2011-01-01,0.548502,NaN,NaN,NaN,NaN
2011-02-01,0.519609,-0.052675,NaN,NaN,NaN
2011-03-01,0.423300,-0.185349,NaN,NaN,NaN
2011-04-01,0.458613,0.083423,NaN,NaN,NaN
2011-05-01,0.459531,0.002000,-0.052675,NaN,NaN
...,...,...,...,...,...
2024-03-01,90.344337,0.142128,0.058841,-0.118651,0.118095
2024-04-01,86.394905,-0.043715,0.242527,-0.062430,0.104767
2024-05-01,109.624001,0.268871,0.285810,0.146886,0.056196


In [6]:
data.dropna(inplace=True)

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
model = LinearRegression()

In [8]:
train,test = train_test_split(data, test_size=0.3, random_state=0)
model.fit(train[lagnames], train['returns'])

# Make predictions for training & testing sets
y_train_pred = model.predict(train[lagnames])
y_test_pred = model.predict(test[lagnames])

# Check if model is good
train_r2 = r2_score(train['returns'], y_train_pred)
test_r2 = r2_score(test['returns'], y_test_pred)

train_r2, test_r2

(0.005988742974078587, -0.2808779863030342)

In [19]:
test['prediction_LR'] = model.predict(test[lagnames])
#test['direction_LR'] = [1 if i > 0 else -1 for i in test.prediction_LR]
#test['strat_LR'] = test['direction_LR'] * test['returns']
#np.exp(test[['returns', 'strat_LR']].sum())
#np.exp(test[['returns', 'strat_LR']].cumsum()).plot()
high_increases = test[test['prediction_LR'] >= 0.03]

In [20]:
print(high_increases.shape)
positive_returns = high_increases[high_increases['returns'] > 0]
positive_returns.shape

(51, 6)


(35, 6)